# Day 12

https://adventofcode.com/2021/day/12

I tried to do something clever to make networkx work this out for me but it didn't work. Instead I did it the hard way, calculating every possible path and storing in a data frame. Part 2 takes about 5 minutes to calculate so there must be a better way (possibly some sort of memoization). 

In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
raw = pd.read_csv("input_12.txt", sep = "-", header = None)

graph_df = raw.rename(columns = {0: "from", 1: "to"})


### Non-working part:

In [5]:
G = nx.Graph()

for i, row in graph_df.iterrows():
    G.add_edge(row["from"], row["to"])
    
G.edges

In [18]:
G = nx.Graph()

for i, row in graph_df.iterrows():
    G.add_edge(row["from"], row["to"])
    
g_nodes = list(G.nodes)
    
for node in g_nodes:
    if not (node.lower() == node):
        ns = list(G.neighbors(node))
        G.remove_node(node)
        new_nodes = []
        for n in ns:
            new_node_name = node + "_" + n
            new_nodes.append(new_node_name)
            G.add_edge(new_node_name, n)
            
        for new_node_1 in new_nodes:
            for new_node_2 in new_nodes:
                if new_node_1 != new_node_2:
                    G.add_edge(new_node_1, new_node_2)


In [19]:
G.edges

EdgeView([('ed', 'eq'), ('ed', 'cc'), ('ed', 'tk'), ('ed', 'XW_ed'), ('ed', 'LA_ed'), ('ed', 'II_ed'), ('cc', 'tk'), ('cc', 'kl'), ('cc', 'II_cc'), ('tk', 'XW_tk'), ('tk', 'LA_tk'), ('tk', 'II_tk'), ('tk', 'SQ_tk'), ('eq', 'ns'), ('eq', 'XW_eq'), ('eq', 'II_eq'), ('eq', 'JR_eq'), ('kl', 'start'), ('kl', 'LA_kl'), ('kl', 'SQ_kl'), ('end', 'XW_end'), ('end', 'LA_end'), ('end', 'II_end'), ('start', 'LA_start'), ('start', 'SQ_start'), ('XW_ed', 'XW_eq'), ('XW_ed', 'XW_tk'), ('XW_ed', 'XW_end'), ('XW_eq', 'XW_tk'), ('XW_eq', 'XW_end'), ('XW_tk', 'XW_end'), ('LA_kl', 'LA_end'), ('LA_kl', 'LA_ed'), ('LA_kl', 'LA_tk'), ('LA_kl', 'LA_start'), ('LA_end', 'LA_ed'), ('LA_end', 'LA_tk'), ('LA_end', 'LA_start'), ('LA_ed', 'LA_tk'), ('LA_ed', 'LA_start'), ('LA_tk', 'LA_start'), ('II_tk', 'II_end'), ('II_tk', 'II_cc'), ('II_tk', 'II_eq'), ('II_tk', 'II_ed'), ('II_end', 'II_cc'), ('II_end', 'II_eq'), ('II_end', 'II_ed'), ('II_cc', 'II_eq'), ('II_cc', 'II_ed'), ('II_eq', 'II_ed'), ('SQ_kl', 'SQ_start'),

In [32]:
paths = nx.all_simple_paths(G, "start", "end")

### Working part:

In [3]:
G = nx.Graph()

for i, row in graph_df.iterrows():
    G.add_edge(row["from"], row["to"])
    
for node in G.nodes:
    G.nodes[node]["big"] = not (node.lower() == node)
    

def paths(G, double_visit = False):
    
    df = pd.DataFrame({"double_visited_cave": [""], "node_0": ["start"]})
    
    while not (df[df.columns[-1]] == "end").all():
        last_col_name = df.columns[-1]
        next_col_id = int(last_col_name.replace("node_", "")) + 1
        next_col_name = "node_" + str(next_col_id)

        next_col = []
        for i, row in df.iterrows():
            node = row[last_col_name]
            if node == "end":
                next_col.append(["end"])
            else:
                ns = [n for n in G.neighbors(node) if n != "start"]
                next_col.append([n for n in ns if G.nodes[n]["big"] or (n not in row.values[1:-1]) or (double_visit and row["double_visited_cave"] == "")])

        df[next_col_name] = next_col
        df = df.explode(next_col_name)
        df = df.dropna(subset = [next_col_name])
        
        if double_visit:
            for i, row in df.iterrows():
                n = row[next_col_name]
                
                if n == "end":
                    continue
                if (not G.nodes[n]["big"]) and (n in row.values[1:-1]):
                    if row["double_visited_cave"] == "":
                        row["double_visited_cave"] = n
                    else:
                        row[next_col_name] = np.nan
        df = df.dropna(subset = [next_col_name])

    return df

In [4]:
df = paths(G, double_visit = False)

In [5]:
df.shape

(3679, 14)

In [6]:
df_2 = paths(G, double_visit = True)

In [7]:
df_2.shape

(107395, 16)